In [2]:
!git clone https://github.com/m19eremeeva/project_NN.git
!pip install rdkit
%cd /content/project_NN

/content/project_NN


In [5]:
import pandas as pd
import torch
from pretrain_trfm import TrfmSeq2seq
from build_vocab import WordVocab
from utils import split

In [ ]:
#(don't run if u already have vocab.pkl in this dir)
#chembl_24_chemreps.txt has too big to upload it to github
#chembl_24_chemreps.txt was taken from https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/releases/chembl_24_1/chembl_24_1_chemreps.txt.gz
df = pd.read_table('chembl_24_chemreps.txt')
df.head()

,chembl_id,canonical_smiles,standard_inchi,standard_inchi_key
0,CHEMBL153534,Cc1cc(cn1C)c2csc(N=C(N)N)n2,InChI=1S/C10H13N5S/c1-6-3-7(4-15(6)2)8-5-16-10...,MFRNFCWYPYSFQQ-UHFFFAOYSA-N
1,CHEMBL440060,CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H...,InChI=1S/C123H212N44O34S/c1-19-63(12)96(164-11...,RSEQNZQKBMRQNM-VRGFNVLHSA-N
2,CHEMBL440245,CCCC[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CCCCN...,InChI=1S/C160H268N50O41/c1-23-27-41-95-134(228...,FTKBTEIKPOYCEX-OZSLQWTKSA-N
3,CHEMBL440249,CC(C)C[C@@H]1NC(=O)CNC(=O)[C@@H](NC(=O)[C@H](N...,InChI=1S/C124H154ClN21O39/c1-57(2)48-81-112(17...,UYSXXKGACMHPIM-KFGDMSGDSA-N
4,CHEMBL405398,Brc1cccc(Nc2ncnc3ccncc23)c1NCCN4CCOCC4,InChI=1S/C19H21BrN6O/c20-15-2-1-3-17(18(15)22-...,VDSXZXJEWIWBCG-UHFFFAOYSA-N


In [ ]:
# deleted smiles longer than 100 characters (don't run if u already have vocab.pkl in this dir)
smiles  = df['canonical_smiles'].values
to_drop = []
for i,sm in enumerate(smiles):
    if len(sm)>100:
        to_drop.append(i)
    if df['chembl_id'][i]=='CHEMBL1201364':
        to_drop.append(i)
print(len(to_drop))

97738


In [ ]:
#saved smiles to csv (don't run if u already have vocab.pkl in this dir)
df_dropped = df.drop(to_drop)
df_dropped = df_dropped.drop(['standard_inchi', 'standard_inchi_key'], axis=1)
L = len(df_dropped)
print(L)
df_dropped.to_csv('chembl24.csv', index = False)

1722297


In [ ]:
#build vocab (don't run if u already have vocab.pkl in this dir)
!python 'build_corpus.py'
!python 'build_vocab.py'

100% 1722297/1722297 [01:45<00:00, 16334.77it/s]
Built a corpus file!
Building Vocab
VOCAB SIZE: 45


In [6]:
pad_index = 0
unk_index = 1
eos_index = 2
sos_index = 3
mask_index = 4

vocab = WordVocab.load_vocab('vocab.pkl')

trfm = TrfmSeq2seq(len(vocab), 256, len(vocab), 4)
trfm.load_state_dict(torch.load('trfm_12_23000.pkl', map_location=torch.device('cpu')))
trfm.eval()
print('Total parameters:', sum(p.numel() for p in trfm.parameters()))

Total parameters: 4245037


In [7]:
def get_inputs(sm):
    seq_len = 220
    sm = sm.split()
    if len(sm)>218:
        print('SMILES is too long ({:d})'.format(len(sm)))
        sm = sm[:109]+sm[-109:]
    ids = [vocab.stoi.get(token, unk_index) for token in sm]
    ids = [sos_index] + ids + [eos_index]
    seg = [1]*len(ids)
    padding = [pad_index]*(seq_len - len(ids))
    ids.extend(padding), seg.extend(padding)
    return ids, seg

def get_array(smiles):
    x_id, x_seg = [], []
    for sm in smiles:
        a,b = get_inputs(sm)
        x_id.append(a)
        x_seg.append(b)
    return torch.tensor(x_id), torch.tensor(x_seg)

In [8]:
#load toxicity data
smiles = pd.read_csv('names_smiles.csv', header = None)[1]
labels = pd.read_csv('names_labels.csv', header = None)[1]

In [9]:
# create embeddings
x_split = [split(sm) for sm in smiles.values]
xid, xseg = get_array(x_split)
X = trfm.encode(torch.t(xid))[:,:256]
print(X.shape)


SMILES is too long (224)
SMILES is too long (260)
SMILES is too long (304)
SMILES is too long (257)
SMILES is too long (284)
SMILES is too long (244)
SMILES is too long (245)
SMILES is too long (280)
SMILES is too long (307)
SMILES is too long (292)
SMILES is too long (325)
SMILES is too long (289)
SMILES is too long (255)
SMILES is too long (257)
SMILES is too long (307)
SMILES is too long (243)
SMILES is too long (234)
SMILES is too long (237)
SMILES is too long (298)
SMILES is too long (309)
SMILES is too long (244)
SMILES is too long (289)
SMILES is too long (224)
SMILES is too long (282)
SMILES is too long (243)
SMILES is too long (282)
SMILES is too long (224)
SMILES is too long (311)
SMILES is too long (378)
SMILES is too long (249)
SMILES is too long (243)
SMILES is too long (260)
SMILES is too long (236)
There are 7697 molecules. It will take a little time.
(7697, 256)


In [15]:
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

y = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# convert to torch tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [16]:
# define the model
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear = nn.Linear(n_input_features, 10)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(10, 1)

  def forward(self, x):
    out = self.linear(x)
    out = self.relu(out)
    out = self.linear2(out)
    y_predicted = torch.sigmoid(out)
    return y_predicted

model = LogisticRegression(X_train.shape[1])

In [17]:
# loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [18]:
# training
num_epochs = 10000
for epoch in range(num_epochs):
  # forward pass and loss
  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)

  # bacward pass
  loss.backward()

  # updates
  optimizer.step()

  # zero gradients
  optimizer.zero_grad()
  if (epoch + 1) % 1000 == 0:
    print(f'epoch: {epoch+1}, loss: {loss.item():.4f}')

epoch: 1000, loss: 0.3247
epoch: 2000, loss: 0.3151
epoch: 3000, loss: 0.3093
epoch: 4000, loss: 0.3045
epoch: 5000, loss: 0.3004
epoch: 6000, loss: 0.2968
epoch: 7000, loss: 0.2935
epoch: 8000, loss: 0.2904
epoch: 9000, loss: 0.2875
epoch: 10000, loss: 0.2847


In [19]:
# accuracy check
with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f'accuracy = {acc:.4f}')

accuracy = 0.8955


In [22]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted_cls, target_names = ['Non-toxic', 'Toxic']))

              precision    recall  f1-score   support

   Non-toxic       0.90      0.99      0.94      1355
       Toxic       0.73      0.21      0.32       185

    accuracy                           0.90      1540
   macro avg       0.82      0.60      0.63      1540
weighted avg       0.88      0.90      0.87      1540

